In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'gproject.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
import re

In [4]:
from articles.models import Article, Pathway
from django.db.models import Exists

In [1]:
has_pathway = Article.objects.all().exclude(pathways=None)
[entry for entry in has_pathway]

SynchronousOnlyOperation: You cannot call this from an async context - use a thread or sync_to_async.

In [5]:
TRAIN_DATA = []

In [7]:
i = 0
for ar in has_pathway:
    abstractText = ar.abstract_text.lower()
    points = []
    for me in ar.pathways.all():
        nameOfPathway = me.name.lower()
        for match in re.finditer(nameOfPathway, abstractText):
            if (match.start(),match.end(), LABEL) not in points:
                points.append((match.start(),match.end(), LABEL))
    TRAIN_DATA.append((abstractText, {"entities": points}))
    i += 1

In [6]:
import json

with open('data.json', 'w') as json_file:
    json.dump(TRAIN_DATA, json_file)

In [7]:
from sklearn.model_selection import ShuffleSplit
import numpy as np
rs = ShuffleSplit(n_splits=1, test_size=.25, random_state=0)

train = []
test = []

for train_index, test_index in rs.split(TRAIN_DATA):
    print(test_index)
    train = np.array(TRAIN_DATA)[train_index.astype(int)]
    test = np.array(TRAIN_DATA)[test_index.astype(int)]

ValueError: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [24]:
# Import and load the spacy model
import spacy
from spacy.training import Example
nlp=spacy.blank("en")

# Getting the ner component
ner=nlp.add_pipe('ner')

examples = []
for text, annots in train:
    examples.append(Example.from_dict(nlp.make_doc(text), annots))
nlp.initialize(lambda: examples)
    
test_examples = []
for text, annots in test:
    test_examples.append(Example.from_dict(nlp.make_doc(text), annots))

C:\Users\omer.davarci\PycharmProjects\MetabolysmAnalysisAPI\venv\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "cardiovascular diseases account for more than 30% ..." with entities "[(666, 685, 'PATHWAY')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\Users\omer.davarci\PycharmProjects\MetabolysmAnalysisAPI\venv\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "bépridil action on oxidative phosphorylations depe..." with entities "[(19, 44, 'PATHWAY'), (377, 402, 'PATHWAY'), (586,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\Users\omer.davarci\PycharmProjects\MetabolysmAnalysisAPI\venv

In [25]:
LABEL = "PATHWAY"
# Add the new label to ner
ner.add_label(LABEL)

# Resume training
optimizer = nlp.resume_training()
move_names = list(ner.move_names)

# List of pipes you want to train
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

# List of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [26]:
# Importing requirements
from spacy.util import minibatch, compounding
import random

# Begin training by disabling other pipeline components
with nlp.disable_pipes(*other_pipes) :

    sizes = compounding(1.0, 4.0, 1.001)
    # Training for 30 iterations     
    for itn in range(30):
    # shuffle examples before training
        random.shuffle(examples)
        # batch up the examples using spaCy's minibatch
        batches = minibatch(examples, size=sizes)
        losses = {}
        for batch in batches:
            # Calling update() over the iteration
            nlp.update(batch, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)

Losses {'ner': 94.66666781902313}
Losses {'ner': 268.691018640995}
Losses {'ner': 453.4196068048477}
Losses {'ner': 547.4198567867279}
Losses {'ner': 787.2688343524933}
Losses {'ner': 962.577000439167}
Losses {'ner': 1160.923519730568}
Losses {'ner': 1316.166685461998}
Losses {'ner': 1450.1715864539146}
Losses {'ner': 1516.6800965368748}
Losses {'ner': 1639.3884814679623}
Losses {'ner': 1702.1071450412273}
Losses {'ner': 1751.992738187313}
Losses {'ner': 1765.9833986330777}
Losses {'ner': 1771.4085011439165}
Losses {'ner': 1776.5551198577377}
Losses {'ner': 1779.0741024157542}
Losses {'ner': 1781.0652945913666}
Losses {'ner': 1783.3182651990971}
Losses {'ner': 1785.3268160010957}
Losses {'ner': 1787.3269740695819}
Losses {'ner': 1789.3272252189104}
Losses {'ner': 1791.3271934233367}
Losses {'ner': 1793.3270435419363}
Losses {'ner': 1795.3269467515909}
Losses {'ner': 1799.3265873419086}
Losses {'ner': 1801.326482552957}
Losses {'ner': 1803.3265050599928}
Losses {'ner': 1805.326524134755

In [69]:
# Output directory
from pathlib import Path
output_dir=Path('/content/')

# Saving the model to the output directory
if not output_dir.exists():
    output_dir.mkdir()
nlp.meta['name'] = 'pathway_ner_model'  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to \content


In [79]:
results = nlp.evaluate(test_examples)
print(results)

{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'ents_p': 0.9777777777777777, 'ents_r': 0.9734513274336283, 'ents_f': 0.975609756097561, 'ents_per_type': {'PATHWAY': {'p': 0.9777777777777777, 'r': 0.9734513274336283, 'f': 0.975609756097561}}, 'speed': 37057.594174891565}


In [ ]:
sentiment mining
pathwayleri verip sense'i bulabiliriz